In [1]:
print("Hello")

Hello


In [2]:
import os
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from pydantic import BaseModel

In [3]:
load_dotenv()  # load environment variables from .env

# --- Environment Variable Checks ---
google_api_key_loaded = os.getenv("GOOGLE_API_KEY") is not None

print(f"Google API Key Loaded: {google_api_key_loaded}")

Google API Key Loaded: True


In [4]:
model = init_chat_model("gemini-2.0-flash", 
                        model_provider="google_genai",
                        temperature=0)
# model = init_chat_model("gemini-1.5-flash", model_provider="google_genai")

response = model.invoke("Hello, world!")
print(response)

content='Hello there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--30d88bf0-f0ce-46d3-9abb-b15b813c5d18-0' usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}}


In [5]:
def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

In [6]:
# Add memory
from langgraph.checkpoint.memory import InMemorySaver
checkpointer = InMemorySaver()

# Structured Output
class WeatherResponse(BaseModel):
    conditions: str

In [7]:
agent = create_react_agent(
    # model="anthropic:claude-3-7-sonnet-latest",
    # model="gemini/gemini-pro",
    model=model,
    tools=[get_weather],
    checkpointer=checkpointer,
    response_format=WeatherResponse
)

In [8]:
# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
print(sf_response)
print("****** structure response is ******\n", sf_response["structured_response"])


{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='2519e92b-c538-4d8f-ae89-74da005c516f'), AIMessage(content='Could you please spell out the full city name?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--b2e74d2f-8a1a-46a4-85ad-5a382bf94d5f-0', usage_metadata={'input_tokens': 20, 'output_tokens': 11, 'total_tokens': 31, 'input_token_details': {'cache_read': 0}})], 'structured_response': WeatherResponse(conditions='San Francisco')}
****** structure response is ******
 conditions='San Francisco'
